In [11]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [12]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [13]:
import os
from pyngrok import ngrok

In [24]:
ngrok.kill()

In [25]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://subcritical-readily-mahalia.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://subcritical-readily-mahalia.ngrok-free.dev


True

In [26]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [27]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [28]:
result = stateful_query("簡介三星電子")
print(result)

三星電子（Samsung Electronics Co., Ltd.）是一家總部位於韓國首爾的跨國電子公司，隸屬於三星集團。它於1969年1月13日由李秉喆創立，現已發展成為全球最大的消費電子產品、晶片和電子元件製造商之一，也是最大的資訊高科技公司之一。三星電子在全球58個國家擁有超過30萬名員工。

該公司的全球總部設於韓國首爾瑞草區的三星城（行政）和京畿道水原市（生產），水原市也是其IT研發與半導體的核心重鎮.

三星電子的業務涵蓋廣泛，主要產品包括：
*   **半導體**：是全球最大的記憶體晶片製造商之一，在DRAM和NAND等快閃記憶體領域佔據領先地位。同時也涉足微控制器、影像感測器、微處理器和晶圓代工等業務. 據報導，三星電子在2024年躍居全球半導體銷售額第一名。
*   **行動電話**：2024年起成為全球最大的手機製造高科技公司。
*   **面板**：三星顯示是市場上最大的面板廠。
*   **電視**：長期以來一直是全球領先的電視供應商，尤其在LCD顯示器和QLED電視技術方面具有優勢.
*   **家電**：生產冰箱、洗衣機和空調等各式家用電器.

三星電子在全球市場上扮演著舉足輕重的角色，其多項產品市佔率位居世界第一. 例如，在2013年，三星智慧型手機的全球市場佔有率達到37.7%. 近年來，三星在人工智慧（AI）晶片領域，特別是高頻寬記憶體（HBM）方面，積極追趕競爭對手，並獲得客戶對其HBM4晶片的高度評價. 在2025年，三星電子和SK海力士大幅擴張市佔率，鞏固了全球記憶體晶片製造龍頭地位.

作為三星集團的核心子公司，三星電子在韓國經濟中佔有重要地位，其品牌價值也持續增長.


In [29]:
result2 = stateful_query("最新旗艦手機是什麼？")
print(result2)

目前三星的最新旗艦手機系列是 **Samsung Galaxy S25 系列**，特別是 **Samsung Galaxy S25 Ultra** 被認為是其2025年的最佳手機。

Samsung Galaxy S25 Ultra 擁有以下主要特色：
*   **頂級性能**：搭載為三星客製化的高通 Snapdragon 8 Elite SoC 處理器。
*   **卓越相機**：配備2億畫素主相機和100倍超高倍變焦，以及5000萬畫素超廣角相機、1000萬畫素3倍與5000萬畫素5倍望遠鏡頭。
*   **出色顯示**：採用6.9吋QHD+ Dynamic AMOLED 2X平面螢幕。
*   **大容量記憶體與儲存**：配備12GB或16GB RAM，以及最高達1TB的儲存空間。
*   **S Pen**：內建S Pen。
*   **人工智慧功能**：搭載進化的Galaxy AI人工智慧功能，包括音訊橡皮擦、自動剪輯、最佳臉部表情等。
*   **電池續航**：擁有出色的電池續航力。
*   **軟體支援**：提供長達7年的軟體更新支援。

此外，三星在摺疊手機方面也有旗艦產品，例如 **Galaxy Z Fold7** 和 **Galaxy Z Flip7**。

值得一提的是，關於 **Galaxy S26 系列**的傳聞已經開始浮出水面，預計將在2026年2月左右登場，並可能在AI效能、相機系統和充電速度上有所突破。


In [30]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)

app = Flask(__name__)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    text = event.message.text
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        if text.startswith('AI '):
            prompt = text[3:]
            reply_text = stateful_query(prompt)
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=reply_text)]
                )
            )

        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=event.message.text),
                        TextMessage(text=event.message.text)]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit


BODY:  {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595028895108432077","quoteToken":"ujgP9UtFn3O-3rdFxmYtBu3zgThjNIaYIap1H-hJ6ZOzQb7ndDDlqf_KXvRq16Fxy2DsNRSUUmGhyxhCAKcVCsYbvqErCAARseqqhAXw_7aoQ182k37Fjh6aH-wrBDIVoGvkwKDPdZSMbHITRw55yA","markAsReadToken":"Zpk_zCKOU-uSEUXhPw_qGEfPg0jQyCqN_isZFY5lxDaYdb5An0ooz0JXq_x0Qzw7OBujx86OhDi-1qSiI2SpR_1anZXavIHb8GRkU7R6eq1138gO-OLDdPiOlaJ73dotNtobbe5E2pAr6JLO1QpZsBGJ-MtIDzyeMrotENIUBuPDNgFrJsJqbTp7rIR7QUh5Lu9mJi1KdcaefylukLn2QQ","text":"AI 介紹一下三星電子(50字以內)"},"webhookEventId":"01KE3FP8DD05KCZFTZS3DB553E","deliveryContext":{"isRedelivery":false},"timestamp":1767496163249,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"532c18c9e9fd4de9bc411606818dd02e","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [04/Jan/2026 03:09:25] "POST / HTTP/1.1" 200 -


BODY:  {"destination":"U76cbdc862e10d4bc5006a4f38d10ef71","events":[{"type":"message","message":{"type":"text","id":"595028933947687083","quoteToken":"z6jPLIMfv0jSlvothsN3Y9t9FndVzMwOyKUEg0YPMMkt8vqpCBSuMIuW_Qupul6-hvUKUP3-V5LuCJIbOGKuiKi3QqHyJ6hWmpHLotwyi8kJ8Yvd7XdnAwjxGnCTb8Lld5hYQE-5sifHYq78xPlsqw","markAsReadToken":"UMDn8qF-lKpJ3lFentJHP4Qtrpv9hkFKJelKhJKjBeT4I9uMe-sHS8hdYNzKMG2P7xXMm3u_MLwOQkV1bfEgtsgLN6Aaj3aunP5Wmp1WA4a2V9KAr_U9I5P028SsX_NAkyM_EFY27jMDWgzu_A16LEjexToWtAIeaCSs7UnYaCqBwuO1AGWp0AEdR_CvNaLpSzsPLMjYfJK4zAAavZCNiA","text":"AI 目前最新旗艦手機是什麼？"},"webhookEventId":"01KE3FPYJSSS4JY4M72R7QSPGG","deliveryContext":{"isRedelivery":false},"timestamp":1767496186396,"source":{"type":"user","userId":"U1dd9c39a8d91da6846f0716244d3eece"},"replyToken":"24794d1a96344c1ea38f3f1d74e9f8de","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [04/Jan/2026 03:09:51] "POST / HTTP/1.1" 200 -
